In [25]:
# # download data
# ! wget https://huggingface.co/datasets/urchade/synthetic-pii-ner-mistral-v1/resolve/main/data.json

In [30]:
import json
import random
import torch
from transformers import Trainer, TrainingArguments, AutoModelForTokenClassification, DebertaV2TokenizerFast
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import sys
sys.path.append("../")
import torch
from _gliner import GLiNERConfig, GLiNER
from _gliner.training import Trainer, TrainingArguments
from _gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from _gliner.utils import load_config_as_namespace
from _gliner.data_processing import WordsSplitter, GLiNERDataset

In [26]:
# train_path = "data.json"

# with open(train_path, "r") as f:
#     data = json.load(f)

# print('Dataset size:', len(data))

# random.shuffle(data)
# print('Dataset is shuffled...')

# train_dataset = data[:int(len(data)*0.9)]
# test_dataset = data[int(len(data)*0.9):]

# print('Dataset is splitted...')
# print(test_dataset[0])


In [32]:
# 모델 로드
# model = GLiNER.from_pretrained('taeminlee/gliner_ko', load_tokenizer=True)
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model_name = "taeminlee/gliner_ko"
model = GLiNER.from_pretrained(model_name)


# 기존 모델의 토크나이저 로드
tokenizer = DebertaV2TokenizerFast.from_pretrained('lighthouse/mdeberta-v3-base-kor-further')

# 데이터셋 로드 (위에서 생성한 JSON 파일을 읽어옴)
with open('train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# 데이터셋 준비
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids = self.tokenizer.convert_tokens_to_ids(item['tokenized_text'])
        ner_tags = item['ner']

        labels = [-100] * len(input_ids)  # 기본값 -100
        

        for tag in ner_tags:
            start, end, label = tag
            labels[start:end+1] = [label] * (end - start + 1)

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

train_dataset = NERDataset(train_data, tokenizer)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/anaconda3/envs/ai/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [34]:
# use it for better performance, it mimics original implementation but it's less memory efficient
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [35]:
# Optional: compile model for faster training
model.to(device)
print("done")

done


In [ ]:
# calculate number of epochs
num_steps = 500
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [16]:
trained_model = GLiNER.from_pretrained("models/checkpoint-100", load_tokenizer=True)

config.json not found in /content/models/checkpoint-100
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
text = """
Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 850 official senior career goals for club and country, making him the top goalscorer of all time.
"""

# Labels for entity prediction
labels = ["Person", "Award"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Cristiano Ronaldo dos Santos Aveiro => Person
Ballon d'Or => Award
UEFA Men's Player of the Year Awards => Award
European Golden Shoes => Award
